In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service

import time
import pandas as pd
import requests
import numpy as np
import csv

pd.set_option('display.max_colwidth', 20)

# Get Links
To Do: 
1. Create a csv file named "Links_MonsterSg" in your folder
2. In the csv file, add in the following headers: 
    - job_title
    - company
    - url

In [ ]:
search_url = "https://www.monster.com.sg/srp/results?query=%22Cloud%20Computing%22&locations=Singapore&searchId=89523e9b-a41c-4c28-ae41-fa370f6a9ac4"

all_df = pd.DataFrame()

In [ ]:
driver = webdriver.Chrome(service=Service("C:/chrome/chromedriver.exe"))


driver.get(search_url)

result_text = driver.find_element(By.XPATH, '//*[@id="srp-main-container"]/div[1]/div[1]/span').text
result_num = result_text.split()[-1]
max_page = int(result_num) // 25
# max_page = 2
        
count = 0

while count < max_page: 
    
    driver.get(search_url)
    
    # scroll to bottom of page
    y = 0
    for timer in range(60):
        driver.execute_script("window.scrollTo(0, "+str(y)+")")
        y += 100  
        time.sleep(0.5)    
    
    next_button = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[5]/div[2]/div[2]/div/div[1]/div/div/div[4]/div/div/div[2]/a[2]/button")
    ActionChains(driver).move_to_element(next_button)

    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "medium")))
    
    # If subscribe modal pops up
    driver.switch_to.active_element
    
    try: 
        driver.find_element(By.ID, "wzrk-cancel").click()
        driver.switch_to.active_element
        
    except: 
        pass 

    # get data
    details_dict = { 'job title': [], 'company' : [], 'url': []}

    # job title & url
    try: 
        title_element = driver.find_elements(By.CLASS_NAME, "medium")
        for i in range(len(title_element)):
            if title_element[i].text != "":
                details_dict['job title'].append(title_element[i].text)

            url_element = title_element[i].find_elements(By.TAG_NAME, "a")
            for j in range(len(url_element)):
                url = url_element[0].get_attribute("href")
                details_dict['url'].append(url)
                break
    except:
        print("*** error: job title & url ***")

    # company name 
    try:
        company_element = driver.find_elements(By.CLASS_NAME, "company-name")
        for i in range(len(company_element)):
            details_dict['company'].append(company_element[i].text)
    except:
        print("*** error: company name *** ")

    df = pd.DataFrame(details_dict)
    
    all_df = all_df.append(df, ignore_index=True)
    
    print(f'page {count+1} done') 
    
    if count % 10 == 0:    
        all_df.to_csv('./Links_MonsterSg.csv', mode='a', index=False, header=False)                                 
        all_df = pd.DataFrame()
    
    # Click next   
    wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[1]/div[5]/div[2]/div[2]/div/div[1]/div/div/div[4]/div/div/div[2]/a[2]/button")))  
    next_button.click()
    
    search_url = driver.current_url
    print(search_url)
    
    count += 1
    
all_df.to_csv('./Links_MonsterSg.csv', mode='a', index=False, header=False)                            

### Drop Duplicates
Companies may repost the same job posting

In [ ]:
links_df = pd.read_csv('./Cloud_All Links_MonsterSg.csv')
links_df.drop_duplicates(inplace=True)
links_df

In [ ]:
links_df.to_csv('Cloud_Links_MonsterSg.csv', index=False)

# Get Post
To Do: 
1. Create a csv file named "Posts_MonsterSg" in your folder
2. In the csv file, add in the following headers: 
    'job_title', 'company', 'salary', 'job_type', 'years_experience', 'tech_stack', 
    'job_description', 'industry', 'function', 'roles', 'last_updated', 'posted_on'

#### Function to get data from each job posting
- Job Title ✔
- Company ✔
- Salary ✔
- Job Type ✔
- Seniority ??
- Years of Experience ✔
- Tech Stack ✔
- Job Description ✔
- Industry ✔
- Function ✔
- Roles ✔
- Last updated ✔
- Posted on ✔

In [ ]:
# Update directory
links_df = pd.read_csv('./Cloud_Links_MonsterSg.csv')
links_df.head()

In [ ]:
## remove search id in url

def clean_url(full_url):
    return (full_url.split('&searchId')[0])

links_df['cleaned_url'] = links_df['url'].apply(lambda x: clean_url(x))
links_df

In [ ]:
def get_details(index, post_url, driver):

    driver.get(post_url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    wait = WebDriverWait(driver, 10)

    # If subscribe modal pops up
    driver.switch_to.active_element

    try: 
        driver.find_element(By.ID, "wzrk-cancel").click()
        driver.switch_to.active_element

    except: 
        pass 


    # Salary
    salary = ""
    try: 
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'package')))
        salary = driver.find_element(By.CLASS_NAME, 'package').text
    except: 
        print(f'*** Error: {index} \n  package \n {post_url}')

    # Job Type
    job_type = ""
    try: 
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'color-grey-light')))
        job_type = driver.find_element(By.CLASS_NAME,'color-grey-light').text
    except: 
        print(f'*** Error: {index} \n  job type  \n {post_url}')

    # Years of Experience
    exp = ""
    try: 
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'exp')))
        exp = driver.find_element(By.CLASS_NAME,'exp').text
    except: 
        print(f'*** Error: {index} \n  experience \n {post_url}')

    # Tech Stack
    tech_stack = ""
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(6) > div > div > div > div:nth-child(4) > div.dt-content > p')))
        tech_stack = driver.find_element(By.CSS_SELECTOR,'#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(6) > div > div > div > div:nth-child(4) > div.dt-content > p').text
    except: 
        print(f'*** Error: {index} \n  tech stack \n {post_url}' )
        
    # Job Description
    job_des = ""
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(2) > div > div > div.card-panel.job-description-content > p')))
        job_des = driver.find_element(By.CSS_SELECTOR,'#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(2) > div > div > div.card-panel.job-description-content > p').text
    except: 
         print(f'*** Error: {index} \n  experience \n {post_url}')
            
    # Industry
    industry = ""
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(6) > div > div > div > div:nth-child(1) > div.dt-content > p')))
        industry = driver.find_element(By.CSS_SELECTOR, '#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(6) > div > div > div > div:nth-child(1) > div.dt-content > p').text
    except: 
        print(f'*** Error: {index} \n  industry \n {post_url}')
        
    # Function
    function = ""
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(6) > div > div > div > div:nth-child(2) > div.dt-content > p')))
        function = driver.find_element(By.CSS_SELECTOR,'#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(6) > div > div > div > div:nth-child(2) > div.dt-content > p').text
    except:  
        print(f'*** Error: {index} \n function \n {post_url}')
        
    # Roles
    role = ""
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(6) > div > div > div > div:nth-child(3) > div.dt-content > p')))
        role = driver.find_element(By.CSS_SELECTOR,'#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(6) > div > div > div > div:nth-child(3) > div.dt-content > p').text
    except: 
        print(f'*** Error: {index} \n role \n {post_url}')
                
    # Last Updated        
    y = 4000
    for timer in range(10):
        driver.execute_script("window.scrollTo(document.body.scrollHeight, "+str(y)+")")
        y -= 200  
        time.sleep(1) 
    
    last_updated = ""    
        
    try: 
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div.last-updated.fs-14.color-grey.mt20 > span:nth-child(1)')))
        last_updated = driver.find_element(By.CSS_SELECTOR,'#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div.last-updated.fs-14.color-grey.mt20 > span:nth-child(1)').text
    
    except:
        print(f'*** Error: {index} \n last updated \n {post_url}')
        
    # Date post
    posted_on = ""
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(1) > div > div.card-footer.apply-footer.no-bdr.pd0.bg-grey > div.posted-update.pl0 > span:nth-child(1)')))
        posted_on = driver.find_element(By.CSS_SELECTOR,'#jobDetailHolder > div > div.job-details-wrapper > div.container.mt20.ui-v3 > div.row.fixed-sb-grid > div.col-md-12.col-lg-9 > div:nth-child(1) > div > div.card-footer.apply-footer.no-bdr.pd0.bg-grey > div.posted-update.pl0 > span:nth-child(1)').text
        posted_on = posted_on.split(': ')[1]
    except: 
        print(f'*** Error: {index} \n  posted on \n {post_url}')

        
    return [salary, job_type, exp, tech_stack, job_des, industry, function, role, last_updated, posted_on]

In [ ]:
final_df = pd.DataFrame(columns = ['job_title', 'company', 'salary', 'job_type', 'years_experience', 'tech_stack', 
                                 'job_description', 'industry', 'function', 'roles', 'last_updated', 'posted_on'])

In [ ]:
# Update directory
driver = webdriver.Chrome(service=Service("C:/chrome/chromedriver.exe"))


# Update start and end index 
# started: 7881
start = 0  
end = 2

count = 0
for i, row in links_df[start : end].iterrows(): 
    title = [row['job_title']]
    company = [row['company']]
    url = row['cleaned_url']
    details = get_details(i, url, driver)
    all_details = title + company + details
    
        
    final_df.loc[count] = all_details
    count += 1

    
    if count % 100 == 0:
        print()
        print(i, ' Done') 
        print()
        final_df.to_csv('./Posts_MonsterSg.csv', mode='a', index=False, header=False)
        final_df = pd.DataFrame(columns = ['job_title', 'company', 'salary', 'job_type', 'years_experience', 'tech_stack', 
                                 'job_description', 'industry', 'function', 'roles', 'last_updated', 'posted_on'])
        count = 0
    
        
final_df.to_csv('./Posts_MonsterSg.csv', mode='a', index=False, header=False)

### If crash halfway, run the code below before changing the start index in the prev cell to where it stopped

In [ ]:
final_df

In [ ]:
# append data to the csv

final_df.to_csv('./Posts_MonsterSg.csv', mode='a', index=False, header=False)

In [ ]:
# reset final_df table 

final_df = pd.DataFrame(columns = ['job_title', 'company', 'salary', 'job_type', 'years_experience', 'tech_stack', 
                                 'job_description', 'industry', 'function', 'roles', 'last_updated', 'posted_on'])

In [ ]:
# set the start index to the following result
print(i)

# Check data

In [ ]:
# Update directory
check_df = pd.read_csv('./Posts_MonsterSg.csv')
check_df

### If first few roles are empty

In [ ]:
check_df.dropna(how='all', inplace=True)

In [ ]:
check_df

In [ ]:
# NOTE: overwrites the current file 
# change the directory & name to the one that you saved

check_df.to_csv('./Posts_MonsterSg.csv', index=False)